# Grover's serach for RNA design

In [80]:
import numpy as np
from math import pi,log2,floor,ceil
from qiskit import *
from qiskit.circuit import *
from qiskit.extensions import *
from qiskit.circuit.library import *
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.quantum_info.operators import Operator
from qiskit.extensions.simulator.snapshot import snapshot
from scipy import optimize
from matplotlib.pyplot import plot,show
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

## 1 - Addition

Let's implement the circuit in [1] modified to add mod 2^n.

In [81]:
#MAJ
q=QuantumRegister(3)
MAJ = QuantumCircuit(q)

MAJ.cnot(q[2],q[1])
MAJ.cnot(q[2],q[0])
MAJ.ccx(q[0],q[1],q[2])

maj = MAJ.to_gate(label='MAJ')

#UMA
q=QuantumRegister(3)
UMA = QuantumCircuit(q)

UMA.ccx(q[0],q[1],q[2])
UMA.cnot(q[2],q[0])
UMA.cnot(q[0],q[1])

uma = UMA.to_gate(label='UMA')

#add
def add_circuit(nb_bits):
    concerved = QuantumRegister(nb_bits, name="=")
    not_concerved = QuantumRegister(nb_bits, name="->")
    initial_carry = QuantumRegister(1)
    add = QuantumCircuit(initial_carry,concerved,not_concerved)
    add.append(maj,[initial_carry,not_concerved[0],concerved[0]])
    for i in range(1,nb_bits):
        add.append(maj,[concerved[i-1],not_concerved[i],concerved[i]])
    for i in range(nb_bits-1,0,-1):
        add.append(uma,[concerved[i-1],not_concerved[i],concerved[i]])
    add.append(uma,[initial_carry,not_concerved[0],concerved[0]])
    return add

def add_gate(nb_bits):
    return add_circuit(nb_bits).to_gate(label="add{}".format(nb_bits))

add_circuit(6).draw()

┌──────┐                                                                »
  q5: ┤0     ├────────────────────────────────────────────────────────────────»
      │      │┌──────┐                                                        »
 =_0: ┤2     ├┤0     ├────────────────────────────────────────────────────────»
      │      ││      │┌──────┐                                                »
 =_1: ┤      ├┤2     ├┤0     ├────────────────────────────────────────────────»
      │      ││      ││      │┌──────┐                                ┌──────┐»
 =_2: ┤      ├┤      ├┤2     ├┤0     ├────────────────────────────────┤0     ├»
      │  MAJ ││      ││      ││      │┌──────┐                ┌──────┐│      │»
 =_3: ┤      ├┤      ├┤      ├┤2     ├┤0     ├────────────────┤0     ├┤2     ├»
      │      ││  MAJ ││      ││      ││      │┌──────┐┌──────┐│      ││      │»
 =_4: ┤      ├┤      ├┤      ├┤      ├┤2     ├┤0     ├┤0     ├┤2     ├┤      ├»
      │      ││      ││  MAJ ││      ││      ││      ││      ││      ││      │»
 =_5: ┤      ├┤      ├┤      ├┤      ├┤      ├┤2     ├┤2     ├┤      ├┤      ├»
      │      ││      ││      ││  MAJ ││      ││      ││      ││      ││  UMA │»
->_0: ┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
      └──────┘│      ││      ││      ││  MAJ ││      ││      ││  UMA ││      │»
->_1: ────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
              └──────┘│      ││      ││      ││  MAJ ││  UMA ││      ││      │»
->_2: ────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤      ├┤      ├»
                      └──────┘│      ││      ││      ││      ││      ││      │»
->_3: ────────────────────────┤1     ├┤      ├┤      ├┤      ├┤      ├┤1     ├»
                              └──────┘│      ││      ││      ││      │└──────┘»
->_4: ────────────────────────────────┤1     ├┤      ├┤      ├┤1     ├────────»
                                      └──────┘│      ││      │└──────┘        »
->_5: ────────────────────────────────────────┤1     ├┤1     ├────────────────»
                                              └──────┘└──────┘                »
«                      ┌──────┐
«  q5: ────────────────┤0     ├
«              ┌──────┐│      │
« =_0: ────────┤0     ├┤2     ├
«      ┌──────┐│      ││      │
« =_1: ┤0     ├┤2     ├┤      ├
«      │      ││      ││      │
« =_2: ┤2     ├┤      ├┤      ├
«      │      ││      ││  UMA │
« =_3: ┤      ├┤      ├┤      ├
«      │      ││  UMA ││      │
« =_4: ┤      ├┤      ├┤      ├
«      │  UMA ││      ││      │
« =_5: ┤      ├┤      ├┤      ├
«      │      ││      ││      │
«->_0: ┤      ├┤      ├┤1     ├
«      │      ││      │└──────┘
«->_1: ┤      ├┤1     ├────────
«      │      │└──────┘        
«->_2: ┤1     ├────────────────
«      └──────┘                
«->_3: ────────────────────────
«                              
«->_4: ────────────────────────
«                              
«->_5: ────────────────────────
«

And also a circuit to increment and a circuit to decrement.

In [82]:
def incr_circuit(nb_bits):
    number = QuantumRegister(nb_bits, name="number")
    incr = QuantumCircuit(number)
    for i in range(nb_bits-1):
        incr.append(MCXGate(nb_bits-i-1),[number[j] for j in range(nb_bits-i)])
    incr.x(number[0])
    return incr

def incr_gate(nb_bits):
    return incr_circuit(nb_bits).to_gate(label="incr{}".format(nb_bits))

def decr_circuit(nb_bits):
    return incr_circuit(nb_bits).inverse()

def decr_gate(nb_bits):
    return decr_circuit(nb_bits).to_gate(label="decr{}".format(nb_bits))

incr_circuit(6).draw()
decr_circuit(6).draw()

┌───┐                         
number_0: ┤ X ├──■────■────■────■────■──
          └───┘┌─┴─┐  │    │    │    │  
number_1: ─────┤ X ├──■────■────■────■──
               └───┘┌─┴─┐  │    │    │  
number_2: ──────────┤ X ├──■────■────■──
                    └───┘┌─┴─┐  │    │  
number_3: ───────────────┤ X ├──■────■──
                         └───┘┌─┴─┐  │  
number_4: ────────────────────┤ X ├──■──
                              └───┘┌─┴─┐
number_5: ─────────────────────────┤ X ├
                                   └───┘

## 2 - Characterise configurations

helper non essential functions

In [119]:
#keywords to find important comments:
#OPTIMISATION: there is (or may be) an optimisation here
#FUTURE: something that should be done if a certain feature was added in the future
#AMPLITUDE: diffrent configurations does not have the same amplitude, so it may be important for the Grover search
#GOOD PRACTICE: this is NOT a reliable way to do, please change it
#TODO

nb_bits_bracket_dot = 2
nb_bits_base = 2
nb_bits_energy = 5 #TODO
min_length_hairpin_loop = 3

def nb_bits_wanna_count_from_zero_until(n):
    return ceil(log2(float(n)))+1

def n_to_ctrl_state(n,nb_bits):
    return format(n,"0{}b".format(nb_bits))[:nb_bits]

#def grey_code_to_n(n_grey_code):
#    return int("".join(reversed(["0" if bit else "1" for bit in n_grey_code])),2)
#
##to be used this way
#
#k_grey_code = [True]*nb_bits_position
#next_k_grey_code = [True]*nb_bits_position
#
#parity = True
#
#for _ in range((2**nb_bits_position)-1):#-1 beacause we avoid 0...0 == next_k_grey_code there [*]
#    #this part is only about dealing with k
#    #what is done once k gets its next value starts there [*]
#
#    #compute what would be the next_k_grey_code 
#    if parity:
#        next_k_grey_code[0] = not next_k_grey_code[0]
#    else:
#        first_one = 0
#        while nb_bits_position > first_one and next_k_grey_code[first_one]:
#            first_one+=1
#        #when 0...0 == next_k_grey_code
#        #then True == parity
#        #so nb_bits_position == first_one should never happend
#        if nb_bits_position-1 == first_one:
#            next_k_grey_code[nb_bits_position-1] = not next_k_grey_code[nb_bits_position-1]
#        else:
#            next_k_grey_code[first_one+1] = not next_k_grey_code[first_one+1]
#    parity = not parity
#
#    k = grey_code_to_n(next_k_grey_code)-1
#    
#    print(next_k_grey_code,k)
#    
#    if condition(k):
#        for i in range(nb_bits_position):
#            if k_grey_code[i] != next_k_grey_code[i]:
#                circuit.x(k_register[i])
#                k_grey_code[i] = next_k_grey_code[i]
#
#        #[*] k_register == k

def base_to_set(base):
    if "N" == base or "." == base:
        return set(("A","C","G","U"))
    elif "A" == base:
        return set(("A"))
    elif "C" == base:
        return set(("C"))
    elif "G" == base:
        return set(("G"))
    elif "U" == base:
        return set(("U"))
    elif "R" == base:
        return set(("A","G"))
    elif "Y" == base:
        return set(("C","U"))
    elif "S" == base:
        return set(("C","G"))
    elif "W" == base:
        return set(("A","U"))
    elif "K" == base:
        return set(("G","U"))
    elif "M" == base:
        return set(("A","C"))
    elif "B" == base:
        return set(("C","G","U"))
    elif "D" == base:
        return set(("A","G","U"))
    elif "H" == base:
        return set(("A","C","U"))
    elif "V" == base:
        return set(("A","C","G"))

def base_pair_complement(base_set):
    complement = set(())
    for base in base_set:
        if "A" == base:
            complement.add("U")
        elif "C" == base:
            complement.add("G")
        elif "G" == base:
            complement.add("C")
        elif "U" == base:
            complement.add("A")
    return complement

def is_valid_base_pair(left,right):
    left = base_to_set(left)
    right = base_to_set(right)
    right = base_pair_complement(right)
    return not left.intersection(right).issubset(set(()))

def bracket_dot_to_ctrl_state(bd):
    if "(" == bd:
        return "01"
    if ")" == bd:
        return "11"
    if "." == bd:
        return "0"
    if "|" == bd:
        return "1"
    if "<-." == bd:
        return "00"
    if ".->" == bd:
        return "10"

def characterise_loop_specification(length,folding_specification,sequence_specification,folding_specification_param,sequence_specification_param):
    sequence_specification_param = (sequence_specification if None == sequence_specification_param else sequence_specification_param)
    
    length = len(folding_specification)
    bottom = [None,None,None,-1,0]
    bottom[0] = bottom
    bottom[1] = bottom
    bottom[2] = bottom
    loop_specification = [[bottom,bottom,bottom,k,0] for k in range(length)]
    
    #semantics of loop_specification
    
    #[0] matching
    #if folding_specification: k_left(-k_right)
    #then, compute loop_specification[k_left][0]=loop_specification[k_right]
    #otherwise let loop_specification[k_left][0]=None
    
    
    #if folding_specification: k_prev(-k_next(
    #or folding_specification: k_prev)-k_next)
    
    #[1] next
    #then, compute loop_specification[k_prev][1]=loop_specification[k_next]
    #otherwise let loop_specification[k_prev][1]=None
    
    #[2] previous
    #then, compute loop_specification[k_next][2]=loop_specification[k_prev]
    #otherwise let loop_specification[k_next][2]=None
    
    #[3] position
    
    #[4] loop length if "(" == folding_specification[k]
    #0 otherwise
    
    open_loop_stack = []
    prev_open = None
    prev_close = None
    for k in range(length):
        if "(" == folding_specification[k]:
            if None != prev_open:
                prev_open[1] = loop_specification[k]
                loop_specification[k][2] = prev_open
            open_loop_stack.append(loop_specification[k])
            prev_open = loop_specification[k]
            prev_close = None
        elif ")" == folding_specification[k]:
            if None != prev_close:
                prev_close[1] = loop_specification[k]
                loop_specification[k][2] = prev_close
            try:
                left = open_loop_stack.pop()
            except IndexError:
                raise Exception(f"Wrong folding specification format: \"{folding_specification_param[k]}\" in folding_specification at index {k} has no matching \"(\".")
            left[0] = loop_specification[k]
            if not is_valid_base_pair(sequence_specification[left[3]],sequence_specification[k]):
                raise Exception(f"Folding specification and sequence specification do not match: "+
                                f"\"{folding_specification_param[left[3]]}\" in folding_specification at index {left[3]} "+
                                f"matches "+
                                f"\"{folding_specification_param[k      ]}\" in folding_specification at index {k}, "+
                                f"but "+
                                f"\"{sequence_specification_param[left[3]]}\" in sequence_specification at index {left[3]} "+
                                f"does not form a valid base pair with "+
                                f"\"{sequence_specification_param[k      ]}\" in sequence_specification at index {k}.")
            prev_close = loop_specification[k]
            prev_open = None
        elif "." == folding_specification[k]:
            if 0 < len(open_loop_stack):
                open_loop_stack[-1][4]+=1
        else:
            raise Exception(f"Wrong folding specification format: \"{folding_specification_param[k]}\" found in folding_specification at index {k}.")
    if 0 < len(open_loop_stack):
        raise Exception(f"Wrong folding specification format: \"{folding_specification_param[open_loop_stack[-1][3]]}\" in folding_specification at index {k} has no matching \")\".")
    return loop_specification

def will_use_registers(circuit,registers):
    for reg in registers:
        if list != type(reg):
            if not circuit.has_register(reg):
                circuit.add_register(reg)
        else:
            for r in reg:
                if not circuit.has_register(r):
                    circuit.add_register(r)


#GOOD PRACTICE: please use classes instead of this!

def to_circuit_builder(append):
    def circuit_builder(folding_specification,sequence_specification=None,*other_args):
        #save parameters for error messages
        folding_specification_param = folding_specification
        sequence_specification_param = sequence_specification
        
        #semantics of sequence_specification
        #follow the IUPAC code
        #R == A or G
        #Y == C or U
        #S == G or C
        #W == A or U
        #K == G or U
        #M == A or C
        #B == C or G or U
        #D == A or G or U
        #H == A or C or U
        #V == A or C or G
        #N == A or C or G or U
        
        #check the input is well formed
        
        folding_specification = str(folding_specification)
        length = len(folding_specification)
        
        if length < min_length_hairpin_loop+2:
            raise Exception(f"folding_specification's length is to small (minimum size is {min_length_hairpin_loop+2}).")
        
        if None == sequence_specification:
            sequence_specification="N"*length
        else:
            sequence_specification=str(sequence_specification).upper()
            if length != len(sequence_specification):
                raise Exception(f"Wrong sequence specification format: sequence_specification's length ({len(sequence_specification_param)}) is different from folding_specification's length ({len(folding_specification_param)}).")
        
        loop_specification = characterise_loop_specification(length,folding_specification,sequence_specification,folding_specification_param,sequence_specification_param)
        
        all_specifiers = [length,sequence_specification,folding_specification,loop_specification]
        
        nb_bits_position = nb_bits_wanna_count_from_zero_until(length) #from 0 until length because we add a ( at the begining to check the folding is well formed
        max_enclosed_loop = (length-2)//(2+min_length_hairpin_loop)
        nb_bits_enclosed = 0 if max_enclosed_loop == 0 else nb_bits_wanna_count_from_zero_until(max_enclosed_loop) #from 0 until the maximum number of loops (of folding (...)) that could be enclosed in a multiloop
        #TODO set nb_bits_incorrect
        nb_bits_incorrect = nb_bits_wanna_count_from_zero_until(length+length)#from 0 because 0==incorrect_configuration when the configuration is correct, length (and not length+1 indeed) is the maximum number of incrementations of incorrect_configuration due to incorrect folding and length is the maximum contribution of incorrect base pairing to incorrect_configuration
        
        all_nb_bits=[nb_bits_position,nb_bits_enclosed,nb_bits_incorrect]
        
        incr_position=incr_gate(nb_bits_position)
        decr_position=decr_gate(nb_bits_position)
        incr_incorrect=incr_gate(nb_bits_incorrect)
        decr_incorrect=decr_gate(nb_bits_incorrect)
        incr_nb_enclosed= (IGate() if 0 == nb_bits_enclosed else incr_gate(nb_bits_enclosed))
        incr_energy=incr_gate(nb_bits_energy)
        decr_energy=decr_gate(nb_bits_energy)
        add_energy=add_gate(nb_bits_energy)
        
        all_gates=[incr_position,decr_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy]
        
        #input
        #semantics of folding
        #<-. == "00"
        #.-> == "10"
        #) == "11"
        #( == "01"
        #so
        #. == "x0"
        #| == "x1"
        folding = [QuantumRegister(nb_bits_bracket_dot,name=f"(/./h/){k}") for k in range(length)]
        #semantics of sequence
        #A == "00"
        #U == "01"
        #G == "10"
        #C == "11"
        sequence = [QuantumRegister(nb_bits_base,name=f"A/U/G/C{k}") for k in range(length)]

        #auxillary qbits
        change_sequence_encoding = QuantumRegister(length,name="change_seq")
        count = QuantumRegister(nb_bits_position,name="count") #count how many parenthesis still need to be closed
        continue_searching_right_matching_left = QuantumRegister(1,name="continue_match")
        nb_enclosed = QuantumRegister(nb_bits_enclosed,name="enclosed") #count how many stems are enclosed in the loop
        loop_length = QuantumRegister(nb_bits_position,name="length") #loop_length is initialised to 1 (actually not 0 because of an optimisation used there <**>) and is incremented each time 1 == count and . == folding until 0 == count
        loop_length_right = QuantumRegister(nb_bits_position,name="length_right")
        #in the order of the sequence:
        next_left_prev = QuantumRegister(nb_bits_base,name="next_left_prev")
        next_left = QuantumRegister(nb_bits_base,name="next_left")
        right_matching_next_left_next = QuantumRegister(nb_bits_base,name="next_right_next")
        right_matching_left_prev = QuantumRegister(nb_bits_base,name="right_prev")
        right_matching_left_next = QuantumRegister(nb_bits_base,name="right_next")
        
        #results
        incorrect_configuration = QuantumRegister(nb_bits_incorrect,name="incorrect")
        energy = QuantumRegister(nb_bits_energy,name="energy_delta")
        
        all_registers=[folding,sequence,change_sequence_encoding,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration,energy]
        
        circuit = QuantumCircuit()

        #TODO
        def semantics(bit_string):
            i_bit = 0
            semantics_string = ""

            def semantics_number(label,nb_bits_number):
                s=str(int(bit_string[i_bit:i_bit+nb_bits_number],2))
                delta=nb_bits_number-len(s)
                return s+"_"+label+delta*" "+" ",nb_bits_number

            if circuit.has_register(correct_folding):
                s,i=semantics_number("?",1)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(incorrect_folding):
                s,i=semantics_number("!",nb_bits_position)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(loop_type):
                s,i=semantics_number("lt",nb_bits_loop_type)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(loop):
                s,i=semantics_number("lo",nb_bits_position)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(continue_scanning):
                s,i=semantics_number("sc",1)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(continue_searching_right_matching_left):
                s,i=semantics_number("se",1)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(zero):
                semantics_string+="cst"
                semantics_string+=bit_string[i_bit:i_bit+1]
                i_bit+=1
            #if circuit.has_register(one):
            #    semantics_string+=bit_string[i_bit:i_bit+1]
            #    i_bit+=1
                semantics_string+=" "
            if circuit.has_register(k_right_matching_left):
                s,i=semantics_number("km",nb_bits_position)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(count):
                s,i=semantics_number("c",nb_bits_position)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(k_count):
                s,i=semantics_number("kc",nb_bits_position)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(change_folding_encoding):
                semantics_string+="-" if bit_string[i_bit:i_bit+1]=="0" else "h"
                i_bit+=1
                semantics_string+=" "
            if circuit.has_register(k_folding):
                s,i=semantics_number("kf",nb_bits_position)
                semantics_string+=s
                i_bit+=i
            if circuit.has_register(folding):
                semantics_string+=("." if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="00" else
                                   "h" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="11" else
                                   "(" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="01" else
                                   ")" if bit_string[i_bit:i_bit+nb_bits_bracket_dot]=="10" else "")
                i_bit+=nb_bits_bracket_dot
            return semantics_string
        
        basic_args_to_call_append=[circuit]+all_registers+[all_registers]+all_nb_bits+[all_nb_bits]+all_gates+[all_gates]+all_specifiers+[all_specifiers]
        basic_args_to_call_append.append(basic_args_to_call_append)
        
        label=append(*basic_args_to_call_append,*other_args)
        return circuit,semantics,label
    return circuit_builder

def to_gate_builder(append):
    def gate_builder(length,*other_args):
        circ,_,lab=to_circuit_builder(append)(length,sequence_specification,folding_specification,
                                              *other_args)
        return circ.to_gate(label=lab)
    return gate_builder

def to_gate_inverse_builder(append):
    def gate_inverse_builder(length,sequence_specification,folding_specification,
                             *other_args):
        circ,_,lab=to_circuit_builder(append)(length,sequence_specification,folding_specification,
                                              *other_args)
        return circ.inverse().to_gate(label=lab+"^-1"),
    return gate_inverse_builder

def inversed(append):
    def append_inversed(circuit,
                        folding,sequence,change_sequence_encoding,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration,energy,
                        all_registers,
                        nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                        all_nb_bits,
                        incr_position,decr_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                        all_gates,
                        length,sequence_specification,folding_specification,loop_specification,
                        all_specifiers,
                        basic_args_to_call_append,
                        *other_args):
        circ = QuantumCircuit()
        #add diffrent QuantumRegister to the circ and append diffrent gates
        basic_args_to_call_append[0]=circ
        lab=append(*basic_args_to_call_append,*other_args)
        basic_args_to_call_append[0]=circuit
        all_registers_list = folding+sequence+[all_registers[i] for i in range(2,len(all_registers))]
        register_list =list(np.asarray(all_registers_list,dtype=Register)[[circ.has_register(reg) for reg in all_registers_list]])
        
        will_use_registers(circuit,register_list)
        qbit_list = [qbit for register in register_list for qbit in register]
        circuit.compose(circ.inverse(),qbit_list)
        return lab+"^-1"
    return append_inversed


important circuit building functions

In [130]:
def automaton_step(circuit,
                   folding,sequence,change_sequence_encoding,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration,energy,
                   all_registers,
                   nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                   all_nb_bits,
                   incr_position,decr_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                   all_gates,
                   length,sequence_specification,folding_specification,loop_specification,
                   all_specifiers,
                   basic_args_to_call_append,
                   k_left,k,
                   compute_energy=True,
                   uncompute=False):
    if compute_energy:
        will_use_registers(circuit,[folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next])
        if not uncompute:
            will_use_registers(circuit,[incorrect_configuration])
    else:
        will_use_registers(circuit,[folding,sequence,count,continue_searching_right_matching_left])
    
    #<*> if it has been checked that there is a matching ) to find,
    #then continue_searching_right_matching_left == 1
    #otherwise continue_searching_right_matching_left == 0
    
    
    #1), 2) and 3) before 4) Update count
    #and
    #5) and 6) after 4) Update count
    #so that things are always controled by 1 == count
    #(which could be used by the compiler for optimisation when implementing sevral time in a row the same MCXGate())
    
    if compute_energy:
        
        
        # 1) Get next_left_prev and next_left
        
        for i in range(nb_bits_base):
            #if 0 < k (which is always the case when compute_energy)
            circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state("(")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k-1][i],next_left_prev[i]])
            circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state("(")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k  ][i],next_left     [i]])
        
        
        # 2) Get right_matching_left_prev and right_matching_left_next
        
        for i in range(nb_bits_base):
            circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k-1][i],right_matching_left_prev[i]])
            
            #FUTURE: if one adds GU pairs, then one base of a valid base pair would not be enough anymore to know which base pair it is
            #so there would be a need for right_matching_left, this way:
            #circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k  ][i],right_matching_left[i]     ])
            
            if length-1 > k:
                circuit.append(MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k+1][i],right_matching_left_next[i]])
        if length-1 == k:
            circuit.append(    MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[ignore_right_matching_left_next])
        
        
        # 3) Check base pairing rules
        
        #FUTURE: if one adds GU pairs, then there would be a right_matching_left
        #so base pairing rules could be checked more efficiently in characterise_loop()
        
        #the base encoding is such that cx(sequence[k_left],sequence[k_right]) leads to
        #"11" == sequence[k_right] if and only if (sequence[k_left],sequence[k_right]) is a valid Watson-Crick base pair
        for i in range(nb_bits_base):
            circuit.cx(sequence[k_left][i],sequence[k][i])
        if not uncompute:
            circuit.append(incr_incorrect.control(1+nb_bits_bracket_dot+nb_bits_position+nb_bits_base,"11"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[sequence[k][i] for i in range(nb_bits_base)]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        #there is actually no need to restore sequence[k]
    
    
    # 4) Update count
    
    #if 1 == continue_searching_right_matching_left
    #then update count for the next candidate
    #OPTIMISATION: use smaller increment and decrement circuits when the maximum value of count enables it (since it never goes negative)
    if 0 == k:
        #then -1 == k_left and ( == folding[k=-1] so 1 == continue_searching_right_matching_left
        #so it is ok not controlling by continue_searching_right_matching_left here
        circuit.append(incr_position.control(  nb_bits_bracket_dot,bracket_dot_to_ctrl_state("(")    ),                                         [folding[k][i] for i in range(nb_bits_bracket_dot)]+[count[i] for i in range(nb_bits_position)])
        circuit.append(decr_position.control(  nb_bits_bracket_dot,bracket_dot_to_ctrl_state(")")    ),                                         [folding[k][i] for i in range(nb_bits_bracket_dot)]+[count[i] for i in range(nb_bits_position)])
    else:
        circuit.append(incr_position.control(1+nb_bits_bracket_dot,bracket_dot_to_ctrl_state("(")+"1"),[continue_searching_right_matching_left]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[count[i] for i in range(nb_bits_position)])
        circuit.append(decr_position.control(1+nb_bits_bracket_dot,bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[count[i] for i in range(nb_bits_position)])
    
    #if 0 == continue_searching_right_matching_left then simply increment count so that it never equals 0 again
    circuit.append(incr_position.control(ctrl_state="0"),[continue_searching_right_matching_left]+[count[i] for i in range(nb_bits_position)])
    
    
    if compute_energy:
        
        
        # 5) Get right_matching_next_left_next
        
        if k < length-1:
            for i in range(nb_bits_base):
                #FUTURE: if one adds GU pairs, then one base of a valid base pair is not enough anymore to know which base pair it is
                #so there would be a need for right_matching_next_left, this way:
                #circuit.append(MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k  ][i],right_matching_next_left[i]     ])
                circuit.append(MCXGate(1+nb_bits_bracket_dot+nb_bits_position+1,"1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][j] for j in range(nb_bits_bracket_dot)]+[sequence[k+1][i],right_matching_next_left_next[i]])
        
        
        # 6) Update nb_enclosed
        
        if 0 < nb_bits_enclosed:
            #OPTIMISATION: use smaller increment circuit when the maximum value of nb_enclose enables it (which depends on min_length_hairpin_loop)
            circuit.append(incr_nb_enclosed.control(1+nb_bits_bracket_dot+nb_bits_position,n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(")")+"1"),[continue_searching_right_matching_left]+[count[j] for j in range(nb_bits_position)]+[folding[k][i] for i in range(nb_bits_bracket_dot)]+[nb_enclosed[i] for i in range(nb_bits_enclosed)])
        
        
        # 7) Update loop_length
        
        #loop_length is the number of . for which 1 == count
        
        #if 1 == continue_searching_right_matching_left and 1 == count and "x0" == . == folding[k]
        #then increment loop_length
        #OPTIMISATION: use smaller increment circuit when the maximum value of loop_length enables it
        if k_left+1 == k:
            #since the initial value 1 == loop_length is known then there is no need to incr_position in order to increment
            #two CXGate() are sufficient
            circuit.append(MCXGate(              (nb_bits_bracket_dot-1)+nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(".")),[folding[k][0]]+[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left,loop_length[0]])
            circuit.cx(loop_length[0],loop_length[1],ctrl_state="0")
        else:
            circuit.append(incr_position.control((nb_bits_bracket_dot-1)+nb_bits_position+1,ctrl_state="1"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(".")),[folding[k][0]]+[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left]+[loop_length[i] for i in range(nb_bits_position)])
        
        
        # 8) Update loop_length_right
        
        #in case of a non multi-loop,
        #(that is 1 => nb_enclosed)
        #loop_length_right is the number of . for which 1 == count after the first enclosed stem (if any)
        
        #if 1 == continue_searching_right_matching_left and 1 == count and "x0" == . == folding[k]
        #and 1 == nb_enclosed (that is 1 == nb_enclosed[0] since 1 => nb_enclosed)
        #then increment loop_length_right
        #OPTIMISATION: use smaller increment circuit when the maximum possible value of loop_length enables it
        if k_left+2+min_length_hairpin_loop < k and 0 < nb_bits_enclosed:
            circuit.append(incr_position.control((nb_bits_bracket_dot-1)+nb_bits_position+2,ctrl_state="11"+n_to_ctrl_state(1,nb_bits_position)+bracket_dot_to_ctrl_state(".")),[folding[k][0]]+[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left,nb_enclosed[0]]+[loop_length_right[i] for i in range(nb_bits_position)])
    
    
    # 9) Update continue_searching_right_matching_left

    #if 0 == count
    #then there is no more matching ) to find
    circuit.append(MCXGate(nb_bits_position,ctrl_state=n_to_ctrl_state(0,nb_bits_position)),[count[i] for i in range(nb_bits_position)]+[continue_searching_right_matching_left])
    
    return "step"


def characterise_loop(circuit,
                      folding,sequence,change_sequence_encoding,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration,energy,
                      all_registers,
                      nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                      all_nb_bits,
                      incr_position,decr_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                      all_gates,
                      length,sequence_specification,folding_specification,loop_specification,
                      all_specifiers,
                      basic_args_to_call_append,
                      k_left,
                      compute_energy=True,uncompute=False):
    if compute_energy:
        will_use_registers(circuit,[folding,sequence,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration])
    else:
        will_use_registers(circuit,[folding,sequence,count,continue_searching_right_matching_left                                                                                                                                                   ,incorrect_configuration])
    
    
    # 1) Initialisation of the automaton at the position k_left
    
    #initialise count with folding[k_left]
    #folding[k_left] -> count
    #. == "x0"       -> 0 == "0...000"
    #do nothing
    #) == "11"       -> 0 == "0...000"
    #do nothing
    #( == "01"       -> 1 == "0...001"
    if 0 <= k_left:
        circuit.ccx(folding[k_left][0],folding[k_left][1],count[0],ctrl_state=bracket_dot_to_ctrl_state("("))
    else:
        circuit.x(count[0])
    
    #initialise continue_searching_right_matching_left=1 when 1 == count
    #indeed, when folding[k_left] == ( (which will be checked by there <*>) then there is initially 1 parenthesis to be closed
    circuit.cx(count[0],continue_searching_right_matching_left)
    
    
    # 2) Compute parameters by scanning the configuration with the automaton
    
    for k in range(k_left+1,length-1):
        #OPTIMISATION: break the automaton into sevral automatons to run sequencially in order to reduce the number of qbits
        automaton_step(*basic_args_to_call_append,k_left,k,compute_energy,uncompute)
    
    #Computed parameters
    #folding[k_left] -> continue_searching_right_matching_left * loop_length
    #(               -> 0 if a matching ) has been found       * number of . (at level 1) in the loop
    #(               -> 1 if a matching ) has not been found   * unspecified
    #. or )          -> 0                                      * 0

    #and:
    #loop_length_right
    #next_left_prev
    #next_left
    #right_matching_next_left_next
    #right_matching_left_prev
    #right_matching_left_next
    
    return "characterise"


def initialise_sequence(circuit,
                        folding,sequence,change_sequence_encoding,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration,energy,
                        all_registers,
                        nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                        all_nb_bits,
                        incr_position,decr_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                        all_gates,
                        length,sequence_specification,folding_specification,loop_specification,
                        all_specifiers,
                        basic_args_to_call_append):
    will_use_registers(circuit,[sequence,change_sequence_encoding])
    
    #initialise according to a sequence_specification
    for k_left in range(length):
        if ")" != folding_specification[k_left]:
            base_specification=sequence_specification[k_left]
            #N == A or C or G or U
            if "N" == base_specification or "." == base_specification:
                circuit.h(sequence[k_left])
            elif "A" == base_specification:
                #A == "00"
                #do nothing
                pass
            #C
            elif "C" == base_specification:
                #C == "01"
                circuit.x(sequence[k_left][0])
            #G
            elif "G" == base_specification:
                #G == "10"
                circuit.x(sequence[k_left][1])
            #U
            elif "U" == base_specification:
                #U == "11"
                circuit.x(sequence[k_left])
            #R == A or G
            elif "R" == base_specification:
                circuit.h(sequence[k_left][1])
            #Y == C or U
            elif "Y" == base_specification:
                circuit.x(sequence[k_left][0])
                circuit.h(sequence[k_left][1])
            #S == G or C
            elif "S" == base_specification:
                circuit.h(sequence[k_left][0])
                circuit.cx(sequence[k_left][0],sequence[k_left][1],ctrl_state="0")
            #W == A or U
            elif "W" == base_specification:
                circuit.h(sequence[k_left][0])
                circuit.cx(sequence[k_left][0],sequence[k_left][1])
            #K == G or U
            elif "K" == base_specification:
                circuit.h(sequence[k_left][0])
                circuit.x(sequence[k_left][1])
            #M == A or C
            elif "M" == base_specification:
                circuit.h(sequence[k_left][0])
            #B == C or G or U
            elif "B" == base_specification:
                #A == "00" -> C == "01"
                circuit.h(sequence[k_left])
                circuit.ccx(sequence[k_left][0],sequence[k_left][1],change_sequence_encoding[k_left],ctrl_state="00")
                circuit.cx(change_sequence_encoding,sequence[k_left][0])
                #AMPLITUDE: the amplitude of C is now twice larger than the amplitude of G or the amplitude of U
            #D == A or G or U
            elif "D" == base_specification:
                #C == "01" -> A == "00"
                circuit.h(sequence[k_left])
                circuit.ccx(sequence[k_left][0],sequence[k_left][1],change_sequence_encoding[k_left],ctrl_state="01")
                circuit.cx(change_sequence_encoding,sequence[k_left][0])
                #AMPLITUDE: the amplitude of A is now twice larger than the amplitude of G or the amplitude of U
            #H == A or C or U
            elif "H" == base_specification:
                #G == "10" -> C == "11"
                circuit.h(sequence[k_left])
                circuit.ccx(sequence[k_left][0],sequence[k_left][1],change_sequence_encoding[k_left],ctrl_state="10")
                circuit.cx(change_sequence_encoding,sequence[k_left][0])
                #AMPLITUDE: the amplitude of C is now twice larger than the amplitude of A or the amplitude of U
            #V == A or C or G
            elif "V" == base_specification:
                #U == "11" -> G == "10"
                circuit.h(sequence[k_left])
                circuit.ccx(sequence[k_left][0],sequence[k_left][1],change_sequence_encoding[k_left],ctrl_state="11")
                circuit.cx(change_sequence_encoding[k_left],sequence[k_left][0])
                #AMPLITUDE: the amplitude of G is now twice larger than the amplitude of A or the amplitude of C
            else:
                raise Exception(f"Wrong sequence specification format: \"{sequence_specification[k_left]}\" found at index {k_left} in sequence_specification.")
            
            #only consider sequences that would respect the base pairing rules with the folding_specification
            if "(" == folding_specification[k_left]:
                for i in range(nb_bits_base):
                    #loop_specification[k_left][0][3] is
                    #                             [3] the position
                    #                          [0] of the matching )
                    #                  [k_left] of ( == folding_specification[k_left]
                    circuit.cx(sequence[k_left][i],sequence[loop_specification[k_left][0][3]][i],ctrl_state="0")
    
    return "init_sequence"


def initialise_folding(circuit,
                       folding,sequence,change_sequence_encoding,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration,energy,
                       all_registers,
                       nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                       all_nb_bits,
                       incr_position,decr_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                       all_gates,
                       length,sequence_specification,folding_specification,loop_specification,
                       all_specifiers,
                       basic_args_to_call_append):
    will_use_registers(circuit,[folding])
    #any bracket-dot symbol at any position
    for k in range(length):
        circuit.h(folding[k])
    
    #imagine a position -1, where folding[k_left=-1] == ( to check if the folding is well formed
    
    return "init_folding"


def add_loop_energy(circuit,
                    folding,sequence,change_sequence_encoding,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration,energy,
                    all_registers,
                    nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
                    all_nb_bits,
                    incr_position,decr_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
                    all_gates,
                    length,sequence_specification,folding_specification,loop_specification,
                    all_specifiers,
                    basic_args_to_call_append,
                    k_left):
    will_use_registers(circuit,[folding,sequence,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,energy])
    
    #TODO
    #use https://rna.urmc.rochester.edu/NNDB/tutorials.html
    #please remember loop_length has been initialised to 1


def RNA_design(circuit,
               folding,sequence,change_sequence_encoding,count,continue_searching_right_matching_left,nb_enclosed,loop_length,loop_length_right,next_left_prev,next_left,right_matching_next_left_next,right_matching_left_prev,right_matching_left_next,incorrect_configuration,energy,
               all_registers,
               nb_bits_position,nb_bits_enclosed,nb_bits_incorrect,
               all_nb_bits,
               incr_position,decr_position,incr_incorrect,decr_incorrect,incr_nb_enclosed,incr_energy,decr_energy,add_energy,
               all_gates,
               length,sequence_specification,folding_specification,loop_specification,
               all_specifiers,
               basic_args_to_call_append):
    will_use_registers(circuit,all_registers)
    
    
    # 0) Ad hoc initialisation
    
    #initialise loop_length=1 because:
    # - then too small hairpin loop_length will be length > 4 == 1+min_length_hairpin_loop so it is easy to check there <**>
    # - when doing it at the beginning, it is done in parallele whith other initialisations
    circuit.x(loop_length[0])
    
    
    # 1) Superposition at the beginning of the Grover search
    
    initialise_sequence(*basic_args_to_call_append)
    initialise_folding(*basic_args_to_call_append)
    
    
    # 2) Check ) matching
    
    #check if there is a folding[k_right] == ) non matched with a folding[k_left] == (
    #that would then match with a ( at an imaginary position folding[k_left=-1]
    #the result is in continue_searching_right_matching_left:
    # 1 == continue_searching_right_matching_left if ( == folding[k_left=-1] has not been matched
    # 0 == continue_searching_right_matching_left otherwise
    
    characterise_loop(*basic_args_to_call_append,-1,False,False)

    #increment incorrect_configuration if necessary
    #(it is necessary to increment since the value of incorrect_configuration is unknown since characterise_loop(uncompute=False) can have changed it)
    circuit.append(incr_incorrect.control(ctrl_state="0"),[continue_searching_right_matching_left]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])

    inversed(characterise_loop)(*basic_args_to_call_append,-1,False,True)
    
    
    # 3) Check ( matching and add energ of loop
    
    for k_left in range(length-1-min_length_hairpin_loop):
        
        
        # 3.1) Compute parameters
        
        characterise_loop(*basic_args_to_call_append,k_left)
        
        
        # 3.2) Check ( matching
        
        #increment incorrect_configuration if folding[k_left] == ( has no matching )
        circuit.append(incr_incorrect.control(),[continue_searching_right_matching_left]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        
        #TODO: set nb_bits_incorrect (with respect to the following comment)
        #sum all the incorrect flags continue_searching_right_matching_left (1 per position) into incorrect_folding
        #remark that if the flag continue_searching_right_matching_left[k_left=0] == 0 (at k_left=0 incorrect beeing 0)
        #then there is a ) somewhere
        #so continue_searching_right_matching_left[k_left=somewhere] == 0
        #so incorrect_folding is not incremented at each position
        #so the sum cannot overflow
        
        
        # 3.3) Check hairpin steric constraint
        
        #<**> too small hairpin loop_length are loop_length > 4 == 1+min_length_hairpin_loop
        #and 2 < nb_bits_position since 2+min_length_hairpin_loop == 5 <= length
        circuit.append(incr_incorrect.control(nb_bits_position-2,n_to_ctrl_state(0,nb_bits_position-2)),[loop_length[i] for i in range(2,nb_bits_position)]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        
        
        # 3.4) Compute and add energy of loop to energy
        
        add_loop_energy(*basic_args_to_call_append,k_left)
        
        
        # 3.4) Restore parameter qbits
        
        inversed(characterise_loop)(*basic_args_to_call_append,k_left)
        
    for k_left in range(length-1-min_length_hairpin_loop,length):
        # 3.2 again) Check ( matching
        circuit.append(incr_incorrect.control(nb_bits_bracket_dot,bracket_dot_to_ctrl_state("(")),[folding[k_left][i] for i in range(nb_bits_bracket_dot)]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
    
    #OPTIMISATION
    #use one less bit for folding[length-min_length_hairpin_loop<=k_left<length] just by considering ./)
    #use one less bit for folding[0<=k_left<=min_length_hairpin_loop] just by considering ./(
    
    
    # 4) Check <-.-> choice
    
    #|.->. and .<-.| are prohibited
    for k_dot in range(1,length-1):
        #if 0 == folding[k_dot][0] it is a .
        #then
        # - if 1 == folding[k_dot][1] it is a .->
        #   then
        #   (a) 1 == folding[k_dot-1][0] it is a |
        #   and
        #   (b) 0 == folding[k_dot+1][0] it is a .
        #   is prohibited
        #   so by flipping (a) and (b) if 1 == folding[k_dot][1]
        #   then one get
        #   (flipped a) 0 == folding[k_dot-1][0]
        #   and
        #   (flipped b) 1 == folding[k_dot+1][0]
        #   is prohibited
        #   that is the same condition as the other case since
        #   (flipped a) == (c)
        #   and
        #   (flipped b) == (d)
        #   
        # - if 0 == folding[k_dot][1] it is a <-.
        #   then
        #   (c) 0 == folding[k_dot-1][0] it is a .
        #   and
        #   (d) 1 == folding[k_dot+1][0] it is a |
        #   is prohibited
        
        #flip
        circuit.cx(folding[k_dot][1],folding[k_dot-1][0])
        circuit.cx(folding[k_dot][1],folding[k_dot+1][0])
        
        #test
        circuit.append(incr_incorrect.control(3*(nb_bits_bracket_dot-1),bracket_dot_to_ctrl_state("|")+bracket_dot_to_ctrl_state(".")+bracket_dot_to_ctrl_state(".")),[folding[k_dot-1][0]]+[folding[k_dot][0]]+[folding[k_dot+1][0]]+[incorrect_configuration[i] for i in range(nb_bits_incorrect)])
        
        #flip back
        circuit.cx(folding[k_dot][1],folding[k_dot-1][0])
        circuit.cx(folding[k_dot][1],folding[k_dot+1][0])
    
    
    # 5) substract the energy of the specification_folding
    
    #TODO: using loop_specification
    
    for k_left in range(length-1-min_length_hairpin_loop):
        if "(" == folding_specification[k_left]:
            k_right = loop_specification[k_left][0][3]
            loop_length_specification = loop_specification[k_left][0][3]
            
            #TODO
    
    
    
    #-----------
    
    
    
    #TODO: Grover search
    
    #if a sequence has a 0<=energy for any folding leading to a not incorrect_configuration
    #then it is a sequence we want!
    
    #now time for Grover search
    
    return "RNA_design"


length = 5
folding_specification="(...)"*(length//5)+"."*(length % 5)

circ,_,_=to_circuit_builder(RNA_design)(folding_specification)

circ.draw()

Exception: folding_specification's length is to small (minimum size is 5).

statistics

In [129]:
print("width {} ; depth {}".format(circ.width(),circ.depth()))

width 99 ; depth 997


simulations

In [17]:
#TODO: do the pretty print semantics() and then do simulations

np.set_printoptions(threshold=np.inf)

farthest_truncation_points = 10
length = 2
folding_specification="(...)"*(length//5)+"."*(length % 5)

backend = Aer.get_backend('statevector_simulator')
circ,sem,_=to_circuit_builder(RNA_design)(folding_specification)
nb_bits = circ.width()
job = execute(circ, backend=backend, shots=1, memory=True)
job_result = job.result()
res=np.asarray(job_result.get_statevector(circ))
state = np.asarray([(sem(n_to_ctrl_state(i,nb_bits)),' ') for i,val in enumerate(res[res>10.0**(-15)])])
print(state)
print("\n")

# Transpile for simulator
#simulator = Aer.get_backend('statevector_simulator')
#circ = transpile(circ, simulator)

# Run and get counts
#result = simulator.run(circ).result()
#result
#counts = result.get_counts(circ)
#plot_histogram(counts, title='Bell-State counts')

NameError: name 'RNA_design_circuit' is not defined